In [1]:
#This project tests the performence of weekday long-short portfolio for speculative stocks. The signal for speculation is ROA.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats as st
import statsmodels.api as sm
import wrds
db = wrds.Connection(wrds_username='fxw174')

Loading library list...
Done


In [3]:
#get the return of two portfolios with decile10 and decile1 of ROA
df=db.raw_sql("""
SELECT
    a.*
    ,b.mktrf
    ,b.smb
    ,b.hml
    ,b.umd
FROM
    (SELECT
        b.date
        
        ,avg(CASE WHEN a.dec=1 THEN b.ret ELSE 0 END) AS ret_1
        ,avg(CASE WHEN a.dec=10 THEN b.ret ELSE 0 END) AS ret_10
        ,sum(CASE WHEN a.dec=1 THEN 1 ELSE 0 END) AS count_1
        ,sum(CASE WHEN a.dec=10 THEN 1 ELSE 0 END) AS count_10

    FROM
        (SELECT
            a.*
            ,NTILE(10) OVER(PARTITION BY a.year ORDER BY a.roa) AS dec
        FROM
            (SELECT
                a.datadate
                ,LEFT(a.cusip,8) AS cusip
                ,a.ni/((a.at+b.at)/2) AS roa
                ,EXTRACT(YEAR FROM a.datadate) AS year

            FROM
                comp.funda AS a,comp.funda AS b
            WHERE
                a.cusip=b.cusip
                and a.datadate>='1960-01-01'
                and EXTRACT(YEAR FROM a.datadate)-1=EXTRACT(YEAR FROM b.datadate)
                and (a.at+b.at)!=0
            ) AS a
        ) AS a
        ,crsp.dsf as b
    WHERE
        a.cusip=b.cusip
        and a.year+1=EXTRACT(YEAR FROM b.date)   
        and (a.dec=1 or a.dec=10)
    GROUP BY b.date
    ) AS a, ff_all.factors_daily AS b
WHERE
    a.date=b.date
""",date_cols='date')

In [4]:
df=df[(df['count_1']>=100) & (df['count_10']>=100)]# only keep the ret that has enough samples.
df['lsret']=df['ret_10']-df['ret_1']# get the long-shor portfolio
df['weekday']=pd.DatetimeIndex(df['date']).weekday# get the weekdays

In [5]:
# get the excess return, CAPM alpha and FFs alpha.
def get_return(df):
    Y=df['lsret']
    X=df['mktrf']
    X=sm.add_constant(X)
    model=sm.OLS(Y,X).fit()
    camp=model.params.const
    t_camp=model.tvalues.const
    X=df[['mktrf','smb','hml']]
    X=sm.add_constant(X)
    model=sm.OLS(Y,X).fit()
    ff3=model.params.const
    t_ff3=model.tvalues.const
    X=df[['mktrf','smb','hml','umd']]
    X=sm.add_constant(X)
    model=sm.OLS(Y,X).fit()
    ff4=model.params.const
    t_ff4=model.tvalues.const
    result=pd.DataFrame({
        'xret':Y.mean()
        ,'t_xret':st.ttest_1samp(Y,0)[0]
        ,'CAMP':camp
        ,'t_CAMP':t_camp
        ,'FF3':ff3
        ,'t_FF3':t_ff3
        ,'FF4':ff4
        ,'t_FF4':t_ff4
    },index=[0])
    return result.iloc[0]

In [6]:
df.groupby('weekday').apply(get_return)

,xret,t_xret,CAMP,t_CAMP,FF3,t_FF3,FF4,t_FF4
weekday,,,,,,,,
0,0.000249,1.896298,0.000391,3.611009,0.000218,2.583193,0.000248,2.949550
1,0.000600,5.379471,0.000482,5.069568,0.000360,4.872777,0.000397,5.367881
2,0.000195,1.736463,0.000007,0.071956,-0.000036,-0.482324,-0.000034,-0.454580
3,0.000061,0.547468,-0.000041,-0.433316,-0.000042,-0.568863,-0.000040,-0.543507
4,-0.000323,-3.164367,-0.000451,-5.000281,-0.000628,-8.282541,-0.000625,-8.240975
